# Introduction
Postopus is a project to explore different software designs for **POST**processing for **O**cto**PUS** runs.

In this notebook we explore the basic functionality of the postopus package.

The notebook assumes that you have Octopus installed and is in your (bash) path.
The notebook is structured as follows:
- **Benzene Example** : We use the Benzene example of Octopus to demonstrate the basic functionality of the postopus package.
   - We run the ground state calculation
   - Explore the data structure of postopus 
   - Analyse and plot the data using a package called x-array
   - Use a package called holoviews for plotting
- **Methane Example** : Run gs and td calculation

Before we start the analysis, we import the packages we need.

In [ ]:
# First import all the python modules we might need for the analysis.
import os
from pathlib import Path
import shutil

import matplotlib.pyplot as plt
import numpy as np
from postopus import Run

%matplotlib inline

# Benzene Example (ground state computation)

## Running the simulation

Octopus takes an input file describing the systems and the simulation parameters.
The input file is called `inp` and is a text file with no extension.
The command `octopus` would then have to be executed in the folder that contains this `inp` file.
Since octopus doesn't allow custom names for the input file, an ideal project structure would look like so:

    ├── archimedean_spiral         # Project folder
    │   └── inp                    # Input file
    ├── benzene         
    │   ├── benzene.xyz            # Geometry file or other supporting files
    │   └── inp
    ├── h-atom
    │   └── inp
    ├── he
    │   └── inp
    ├── methane                     # Project folder in case of a multi-stage calculation
    │   ├── calculation_gs          # Ground state calculation
    │   │   └── inp
    │   ├── calculation_td          # Time dependent calculation
    │   │   └── inp
    │   └── inp                     # Input file for the whole calculation (The other files must be placed here one by one in each stage)
    └── recipe
        └── inp
To run one such simulation project one would run the command `octopus` in the root of the folder.
To begin with, lets try to run the benzene example.

First we save the directory path in a variable called `example_dir` and then load it on to python

In [ ]:
example_dir = "examples/benzene"
# Path to some example data
path_to_octopus_output = Path(example_dir)

We now need to run the simulation.
As mentioned before, this involves two steps:
1. Change the directory to the project folder ( that contains the `inp` file )
2. Run the command `octopus` in the terminal ( optionally store the octopus output in a log file by calling `octopus > out_gs.log 2>&1` )


In [ ]:
cd examples/benzene

In [ ]:
!octopus > out_gs.log 2>&1  # Run octopus with benzene example as input and log the output 

We can now look at the log file (if created) to see if the simulation ran successfully.
The log file is stored in the `example_dir` (project) folder.

In [ ]:
!head -n 20 out_gs.log  # Just to see the first 20 lines of the octopus output

In [ ]:
!tail -n 20 out_gs.log  # Just to see the last 20 lines of the octopus output

One would look at the last 20 lines to check if the job ran successfully or if there was any warning. A typical successful output looks like:

        Calculation ended on 2022/10/06 at 15:56:42

                                Walltime:  01.631s

Octopus stores the calculation results in the project directory.
If you were to take a look at the `benzene` project folder, you would see some thing like:

    ├── benzene.xyz                     # Geometry of the molecules ( input )
    ├── exec                            # Runtime information
    │   ├── initial_coordinates.xyz
    │   ├── messages
    │   ├── oct-status-finished
    │   └── parser.log                  # Full set of variables used for the run ( as parsed from the inp file and the code defaults if not specified )
    ├── inp                             # Our input file 
    ├── out_gs.log                      # Log file
    ├── output_iter                     # Output for each iteration in td calculation
    ├── restart
    │   └── gs                          # Ground state calculations
    │       ├── 0000000001.obf          # Checkpoint file to restart calculation incase of job abortion
    │       ├── 0000000002.obf
    │       ├── ...
    │       └── wfns
    └── static                          # Ground state observables are stored here
        ├── info                        # Result of ground state calculation like (energy eigenvalues and forces)
        ├── convergence
        ├── density.cube
        ├── ...
        └── wf-st0015.z=0

These new files are the results of the simulation.
One would now try to analyse the output files to determine their result the postopus package helps in this part.

In [ ]:
cd ../..  

## Postopus project structure

Once the simulation is complete, we can use the postopus package to analyse the data.

We first let postopus know the path to the project folder (stored in the `examples/benzene` directory),
by passing this to a `Run` constructor. Postopus scans through the project folder and creates a object called `run` in our example that contains (nearly) all the information about the simulation:

In [ ]:
run = Run('examples/benzene')

### System selection

Octopus supports the simulation of multiple types of systems, like e.g. `Maxwell`. The system type will normally be set in the `inp` file. If no system is specfically set, postopus will create a default system called `default`. Note: if the systems are explicitly defined, a postopus object will still have a `default` system to hold internal variables. To know all the systems our project has we run:

In [ ]:
run.systems.keys()

Instead of `run.system.keys()` one can also use `run.systems` to get a similar output.

### Calculation modes and subsystems

We can take a look at the different types of calculations done in this system by running:

In [ ]:
run.default.system_data.keys()

This tells us that this project had only a self-consistent field simulation (`scf`).

### Autocompletion

We can take a look at the different fields available for this system by the tab completion feature available in Jupyter notebooks and IPython.
 
Here is an example image of how the tab completion feature works:

![../../images/tab_completion_eg.png](../../images/tab_completion_eg.png)

Instead of using tab completion and if desired, one could use the following command to create a nicely formatted list of the attributes:

In [ ]:
[ attribute for attribute in dir(run.default.scf) if not attribute.startswith('_') ]

(Do not worry if the Python syntax used doesn't make sense to you - the tab completion is generally the convenient way to explore what data is accesible through postopus.)

## Output files

Postopus has many ways to handle octopus' output files. The handling depends on the type of file. We can have:
- unstructured text files like `info`, without any extension. These are returned as they are in a string format.
- table-like-structured files without extension, like `forces`. These are returned as `pandas Dataframes`.
- Extension files, e.g. `xsf` or `cube` files. These are accessed via `get` methods.

### Unstructured text files

We can take a look at the `info` file, which stores information about this calculation by running:

In [ ]:
run.default.scf.info

This is essentially the contents of the file `static/info` that octopus writes.
One can have a look at its output to determine the total energy of the system and the eigenvalues.

One can also look at the forces output from this info by running 

        run.default.scf.forces
        
Typically small forces (`total_x`,`total_y`,`total_z`) indicate a relaxed molecular or solid state structure, otherwise the geometry is perhaps not yet relaxed. 
In such case one might want to use the calculation mode `go` for geometry optimization.

In [ ]:
run.default.scf.forces

### Table-like-structured files 

These files are usually stored in the `static` folder for scf calculations or in the `td.general` folder for time-dependent calculations. These files usually contain physical data that either is one dimensional or without a reference to the coordinate system. Technically, the data provided in rows are presented as a pandas Dataframe object, and we can use all the methods available in pandas to explore them. 

We can take a look at the convergence of the systems energy after each scf iteration through:

In [ ]:
run.default.scf.convergence

If we wanted to visualize the convergence of the energy, we can run the following:

In [ ]:
run.default.scf.convergence['energy'].plot(style='*-' )

These kind of files always have an `.attrs` attribute, which holds further metadata that was stored initially on the file. In this case there is not much stored here appart from the filename, that could be used for setting the plot title, e.g. Nonetheless, in other cases, one can find useful information here.

In [ ]:
run.default.scf.convergence.attrs

### Extension files

These files store field data that have a reference to the coordinate system. They can be stored in the `static` folder (normally converged scf iterations), or in the `output_iter` folder. We will call them Fields from now on. Postopus internally handles them as `ScalarField`s or `VectorField`s.

#### Fields

Fields are quantities that are available for different positions in space, or even time, such as an electric field or an electron density. Often, these are presented as arrays with multiple indices - for example indices for the x, y and z direction for a spatially resolved field.

In our benzene example, we have the following fields: `density`,`elf_rs`, `v0`,`vh`, `vks`, `wf`.

Field data can be accessed with a call to the `get()` method of the required field object.
`get()` takes two parameters, `step` for the iteration number(s) you want to access and `source` for the source of data. For the common scenario that the relevant data is not stored in multiple file formats, we can omit the `source` argument.

Lets look at density as an example:

In [ ]:
density_data = run.default.scf.density.get(steps=1)
density_data

If you want to access the data for all the iterations, you can use `get_all()` or if you only wanted the converged iteration, you can use `get_converged()`. In this specific case, both calls would deliver the same output, since in the benzene `inp` file the `OutputDuringSCF` parameter is not set to `True`, so that we only can access to the last converged iteration. For these last two methods, the user doesn't need to provide a `steps` method.

The data object is an `xarray` object containing not only the data values, but also the coordinates for which these values are defined:

In [ ]:
density_data.coords

In [ ]:
density_data.values.shape

The xarray object holds the metadata needed to search for and visualize the data because it is aware of the coordinates (and the names of the coordinates, such as x, y, and z).  This can make the plotting of the data easier. It also helps selecting subsets of the data. The following section will examine the plotting workflow.

In summary we can conclude the overall data structure as having the following syntax:

 - Scalar fields: `run.systemname.calculationmode.fieldname`
 - Vector fields: `run.systemname.calculationmode.fieldname(.dimension)`

We use the `get` method (or its variants) to access the data.

## Working with the data: x-array

In the previous sections, we saw how to  access any scalar or vector field generated by our calculation.
Let's explore the typical workflow of analyzing the data (slicing and plotting).
First, let's load the density data:

In [ ]:
# ask postopus for the x-array representation of the data
density = run.default.scf.density
xa = density.get_converged()  # the converged field from the "static" folder is loaded.
xa

### Selecting a slice by index and visualizing it

In the first line of the previous cell, we had an overview of the xarray data inside the `density` field.
The first line tells us the number of sampling points in each direction:

    xarray.DataArray ‘density’ (step: 1, x: 95, y: 99, z: 67)

So there are 95 sampling points in the x-direction and similarly 67 for the z-direction.
Suppose we want to have a view of density of benzene in the x-y plane (i.e. at $z=0$), 
we can do so by selecting the slice at z=0. One way to do this for this particular simulation is by asking  xarray for selecting the slice at the index $i_z=67/2~\approx~33$:


In [ ]:
s0 = xa.isel(z=33)  # Viewing the slice at 33rd index of z-axis
s0

This slice returns another xarray object. Note here that the coordinate value for `z` is `1.588e-06`, which is the value of `z` coordinate for the 33rd sampling point in the z-direction.
We can now plot this slice using the `plot()` method of the xarray object:

In [ ]:
s0.plot()  

Note that the plot has the x and the y axis inverted, one can change this by passing the `x='x'` argument to the `plot()` method.

In [ ]:
s0.plot(x='x')  

Another way to slice the data is by using the `sel()` method of the xarray object, where you can pass the *coordinate value* instead of the *index*.

For example, to get the same slice as above, we can use:

In [ ]:
s1 = xa.sel(z=0, method="nearest")
s1.plot(x='x')

Note that plots automatically display the value of the position of the slice in the z-direction as well as the step number. 
This is possible because xarray maintains the metadata of the coordinates even after slicing.

One can have a side view of the molecule by slicing the data in the y-z plane (i.e. at $x=0$) in a similar fashion:

    xa.sel(x=0, method='nearest')

Using a `for` loop and some commands from the `matplotlib` plotting library, we can plot 6 different slices of the data in a 3x2 grid at different values of x.

In [ ]:
# plot of 6 2D slice of the density along the x axis
fig, axs = plt.subplots(2, 3, figsize=(10, 5))          # create a subplot with 2 rows, 3 columns
x_positions = np.linspace(-7.5, 5, 6)                   # x positions from -7.5 to 5 in 6 steps 
for ax, x in zip(axs.flat, x_positions):
    xa.sel(x=x, method="nearest").plot(ax=ax,x='y')     # plot the slice nearest to x position
    
fig.tight_layout()                                      # avoid overlap of labels

The `sel` method can also be used to select a multiple coordinates at once, for example to select the slice at $x=0$ and $z=0$ we can do:

    xa.sel(x=0, z=0, method='nearest')

We can then plot the variation of density along the y axis. Because we have restricted two (`x` and `z`) of the three spatial dimensions, we obtain a 'line plot' of the density along the remaining coordinate dimension `y`.

In [ ]:
xa.sel(x=0, z=0, method="nearest").plot() 

A more detailed exploration of plotting with x-array is available in the [postopus plotting tutorial](https://octopus-code.gitlab.io/postopus/notebooks/xarray-plots1.html#).

## Plotting with holoviews

A plotting library that is built for interactive/multidimensional plots is [holoviews](http://holoviews.org/).
Converting xarray objects (the most common output type in postopus) to holoviews objects is simple, as we will see. Holoviews supports many backends for the plotting. In this tutorial we will use matplotlib and bokeh (the default plotting backends for holoviews).

The holoviews approach to plotting may seem unconventional (and perhaps confusing) at first -- it is, however, very powerful. In particular, holoviews allows the selective and/or interactive visualization of data that is defined in more dimensions that we can easily visualize. To be able to use that power, we need to provide some additional information to the plotting commands: for example which of the many dimensions we want to plot, and for which we would like to have an interactive slider to select it.

We provide a few examples that can be used as templates to cover typical use cases below. 


First, let us import the holoviews library as well as the plotting backends we will use:

In [ ]:
import holoviews as hv
from holoviews import opts  # For setting defaults
# hv.extension('bokeh', 'matplotlib')  # Allow for interactive plots
hv.extension('bokeh')  # Allow for interactive plots

We then choose the color map we want to use for the plots. We use `viridis`.

In [ ]:
# Choose color scheme similar to matplotlib
opts.defaults(
    opts.Image(cmap='viridis', width=400, height=400),
)

We create the object `xa`, which is an xarray of the converged density of the benzene molecule and which we used already earlier in this notebook.

In [ ]:
run = Run("examples/benzene")
xa = run.default.scf.density.get_converged()

We first convert the xarray object `xa` to a holoviews object using the `hv.Dataset` method.

        hv.Dataset(xa)
        
We can now visualise the data as an `hv.Image` using the `to` method of the holoviews object: 
    
        hv.Dataset(xa).to(hv.Image, ['x', 'y'])

We specify the 'x' and 'y' dimensions as the Image's two **k**ey **dim**ensions (kdims) (in this case the `x` and `y` coordinates of the Dataset) that will serve as the visible axes. The remaining dimensions (in this case `z` and `step`) will be used as controls (in the form of slider and a dropdown list) to select the data to display.

The slider for `z` allows us to select the slice at different values of `z` interactively and the dropdown for `step` allows us to select the data at different iterations. (There is only data for one Step in this example.). One can use the `opts` arguments to customize the plot further, for more details see the [holoviews tutorial](https://octopus-code.gitlab.io/postopus/notebooks/holoviews_with_postopus.html).


In [ ]:
hv.Dataset(xa).to(hv.Image, kdims=["x", "y"]).opts(colorbar=True)

Specifying the argument `dynamic=True` in the `to` method allows us to select the data one by one instead of precomputing for the entire range. This may be necessary for larger data points.

Here is a similar plot for the y-z plane (with a slider for x):

In [ ]:
hv.Dataset(xa).to(hv.Image, kdims=["y", "z"], dynamic=True).opts(colorbar=True)

One can also have a carousel of images for these slices instead of a slider, by using the `grid("z")` method.

In [ ]:
plot = hv.Dataset(xa).to(hv.Image, kdims=["y", "x"])
plot.grid("z")

A more detailed exploration of plotting with holoviews is available in the [postopus plotting tutorial](https://octopus-code.gitlab.io/postopus/notebooks/holoviews_with_postopus.html), or in the [holoviews documentation](https://holoviews.org/getting_started/Gridded_Datasets.html).

# Methane Example with time-dependent data

In [ ]:
# first run ground state calculations by
# copying the inp from example_di/calculation_gs to example_dir and run octopus
example_dir = "examples/methane"
shutil.copyfile(
    os.path.join(example_dir, "calculation_gs", "inp"),
    os.path.join(example_dir, "inp")
)
! cd {example_dir}  && octopus > out_gs.log 2>&1 
! cd {example_dir}  && head -n 20 out_gs.log  # Just to see the first 20 lines of the octopus output

In [ ]:
# then run the td calculations by
# copying the inp from example_di/calculation_td to example_dir and run octopus
shutil.copyfile(
    os.path.join(example_dir, "calculation_td", "inp"),
    os.path.join(example_dir, "inp")
)

! cd {example_dir}  && octopus > out_td.log  2>&1
! cd {example_dir}  && head -n 20 out_td.log # Just to see the first 20 lines of the octopus output

In [ ]:
run = Run("examples/methane")

In [ ]:
density = run.default.td.density.get_all(source='ncdf')

In [ ]:
density.coords

Note that now the density xarray has four dimensions we can index: time, x, y, and z.

In [ ]:
density.values.shape

Plot the first time slice, the choose the spatial slice closest to $z=0$:

In [ ]:
density.isel(t=0).sel(z=0, method='nearest').plot()

Plot the density along the y-direction, nearest to $z=0$ and $x=0$, for all time steps:

In [ ]:
density.sel(z=0, method='nearest').sel(x=0, method='nearest').plot()

Plot the density closest to $\vec{r}= (0, 1, 0)$ as a function of time $t$:

In [ ]:
density.sel(x=0, method='nearest').sel(y=0, method='nearest').sel(z=0, method='nearest').plot()

Using holoviews, we can have a slider for time and a spatial slice. For example:

In [ ]:
hv.Dataset(density).to(hv.Image, kdims=["x", "y"], dynamic=True).opts(colorbar=True)